In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import torch
import pickle

f = open("/content/drive/MyDrive/Colab Notebooks/djúp/lokaverkefni/training_data_2019_data_100000_50.dat", "rb")
seen_players, unseen_players, X_seen_train, y_seen_train, X_seen_val, y_seen_val, X_seen_for_embeds, y_seen_for_embeds, X_seen_test, y_seen_test, X_unseen_for_embeds, y_unseen_for_embeds, X_unseen_test, y_unseen_test = pickle.load(f)

In [3]:
unseen_players.shape, X_unseen_for_embeds.shape, y_unseen_for_embeds.shape, X_unseen_test.shape, y_unseen_test.shape

((1870,), (93500, 40), (93500, 2), (93500, 40), (93500, 2))

In [4]:
def get_embeddings(unseen_players, X_unseen_for_embeds, y_unseen_for_embeds, binarizer, N):
  centr_list = []
  corrisponding_player = []
  for player in unseen_players:
    their_white_games = X_unseen_for_embeds[np.logical_and(y_unseen_for_embeds[:,0] == player, y_unseen_for_embeds[:,1] == 1)][:,0:(2*N):2]
    their_black_games = X_unseen_for_embeds[np.logical_and(y_unseen_for_embeds[:,1] == player, y_unseen_for_embeds[:,1] == 0)][:,1:(2*N):2]
    their_games = np.vstack([their_white_games, their_black_games])
    shape = their_games.shape
    their_games_oh = binarizer.transform(their_games.flatten()).reshape(*shape, -1) # We transform here to save RAM
    five_hot = np.sum(their_games_oh, axis = 1)
    emb = np.array(np.mean(five_hot, axis = 0)).flatten()
    centr_list.append(emb)
    corrisponding_player.append(player)
  return np.array(centr_list), np.array(corrisponding_player)

In [5]:
def inference(unseen_players, X_unseen_test, y_unseen_test, neigh, binarizer, N):
  results = []
  for player in unseen_players:
    their_white_games = X_unseen_test[np.logical_and(y_unseen_test[:,0] == player, y_unseen_test[:,1] == 1)][:,0:(2*N):2]
    their_black_games = X_unseen_test[np.logical_and(y_unseen_test[:,0] == player, y_unseen_test[:,1] == 0)][:,1:(2*N):2]
    their_games = np.vstack([their_white_games, their_black_games])
    shape = their_games.shape
    their_games_oh = binarizer.transform(their_games.flatten()).reshape(*shape, -1)
    five_hot = np.sum(their_games_oh, axis = 1)
    y_pred = neigh.predict(five_hot)
    results.append(y_pred == player)
  return np.concatenate(results)

# Running the tests

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer

Ns = [5,10,15,20]
for N in Ns:
    tal = LabelBinarizer()
    tal.fit(X_unseen_for_embeds.flatten())
    X, y = get_embeddings(unseen_players, X_unseen_for_embeds,y_unseen_for_embeds, tal, N)
    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(X, y)
    results = inference(unseen_players, X_unseen_test, y_unseen_test, neigh, tal, N)
    print(f"N={N}:")
    print("="*20)
    print("Unseen players")
    print("Accuracy of the baseline model: ",np.mean(results))
    print("Accuracy using random guess: ", 1/len(unseen_players))
    print("Ratio: ", np.mean(results)*len(unseen_players))
    print()   
    tal = LabelBinarizer()
    tal.fit(X_seen_for_embeds.flatten())
    X, y = get_embeddings(seen_players, X_seen_for_embeds,y_seen_for_embeds, tal, N)
    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(X, y)
    results = inference(seen_players, X_seen_test, y_seen_test, neigh, tal, N)
    print("Seen players")
    print("Accuracy of the baseline model: ",np.mean(results))
    print("Accuracy using random guess: ", 1/len(seen_players))
    print("Ratio: ", np.mean(results)*len(seen_players))
    print("_"*50)


N=5:
Unseen players
Accuracy of the baseline model:  0.01544385026737968
Accuracy using random guess:  0.0005347593582887701
Ratio:  28.88

Seen players
Accuracy of the baseline model:  0.03795
Accuracy using random guess:  0.0025
Ratio:  15.18
__________________________________________________
N=10:
Unseen players
Accuracy of the baseline model:  0.016684491978609627
Accuracy using random guess:  0.0005347593582887701
Ratio:  31.200000000000003

Seen players
Accuracy of the baseline model:  0.0329
Accuracy using random guess:  0.0025
Ratio:  13.16
__________________________________________________
N=15:
Unseen players
Accuracy of the baseline model:  0.01576470588235294
Accuracy using random guess:  0.0005347593582887701
Ratio:  29.48

Seen players
Accuracy of the baseline model:  0.0303
Accuracy using random guess:  0.0025
Ratio:  12.120000000000001
__________________________________________________
N=20:
Unseen players
Accuracy of the baseline model:  0.014406417112299466
Accuracy u